In [1]:
%load Sql.sas

In [ ]:
/*Description of the columns of a specific table (in the log)*/

proc sql;
	describe table sq.customer;
quit;

/*Exercise s101a04 on OBS, INOBS*/
options nolabel;  /*DIsplay in the output the column name, not the labels*/
proc sql;
select FirstName, LastName,State
from sq.customer(obs =10);  /*take the first 10 lines from the input table*/
quit;

proc sql inobs=10 number;  /*As before but  put a warning in the log*/
select FirstName, LastName,State
from sq.customer
quit;

options label;/*re-setup labels*/

/*Exercise s102a01 with OR and IN operators*/

proc sql number;
select FirstName, LastName, State
from sq.customer
where State = "WI" or STATE = "WA";
quit;

proc sql number;
select FirstName, LastName, State
from sq.customer 
where state in ("WI","WA"); /**equivalent to WHERE with OR of previous query*/
quit;

/*Exercise s102a02 on managing NULL values*/

proc sql number;
select FirstName, LastName,CustomerID, CreditScore 
from sq.customer
where creditscore <500 and creditscore is not null; /*ANSI command*/
quit;                                  /*is not missing   SAS command*/
                                       /*ne .*/

/*Exercise s102a03 on ORDER BY clause*/

proc sql number;
select FirstName,LastName, CUstomerID, CreditScore /*also if LastName is not in select data are orderred by lastname*/
from sq.customer 
where creditscore >830
order by Creditscore desc, LastName;  /*(by default ascending)*/
quit;

/*Exercise s102a04 on FORMAT and LABEL clause*/

proc sql number;
select FirstName, LastName, State, UserID label="Email Adress", 
	   Income label="Estimated Income" format=dollar10.2, DOB label="Date of Birth" format=date9.  /*DOB is a date-->original version is a number*/
from sq.customer
where state = "HI" and BankID is not NULL
order by Income desc;
quit;

/*Exercise s102d01 on DATE in where clause*/

title "Top 10 Customers by Income without a BankID and CreditScore Over 700";
title2 "Marketing Report";
proc sql outobs = 10 ;
select FirstName, LastName, State, 
       Income format=dollar16., UserID label="Email"
    from sq.customer 
    where BankID eq . and CreditScore>700
    order by Income desc;
quit;
title;

/*--------*/

title "DOB Prior to December 31, 1940";
title2 "Retirement Campaign";
proc sql;
select CustomerID, State, Zip format z4., 
       DOB label="Date of Birth" format = date9., UserID, 
       HomePhone, CellPhone
    from sq.customer 
    where DOB < "31DEC1940"d and Employed = "Y"
	order by DOB desc;
quit;
title;

/*Exercise s102a05 on calculated columns*/

proc sql outobs=100;
select FirstName, LastName, UserID,DOB label="Date of Birth" format=date9.,round(yrdif(DOB, '01jan2019'd)) as age
from sq.customer
having age >= 70; /*give an error...you must use having on calculated columns!!!*/
quit;

/*Exercise s102d02 on CASE clause*/

proc sql;
select FirstName, LastName, State, CreditScore,
	   case
           when CreditScore >=750 then "Excellent"
	       /*Complete the Simple CASE Expression*/
		   when CreditScore >=700 and CreditScore<=749 then "Good" 
		   when CreditScore >= 650 and CreditScore<=699 then "Fair"
		   when CreditScore >= 550 and CreditScore<=649 then "Poor"
		   when CreditScore<=549 then "Bad"
		   else "Unknown"
	   end as CreditCategory
	from sq.customer
	where CreditCategory = "Excellent"; /*give an error, we need a having clause!!*/
quit;

/*-------------------*/

proc sql;
select FirstName, LastName, State, CreditScore, Married,
       case Married
            when "M" then "Married"
 		    /*Complete the CASE-OPERAND Form*/
            when "D" then "Divorced"
			when "S" then "Single"
			when "W" then "Windowed"
			else "Unknown"
	   end as MarriedCategory
    from sq.customer(obs=1000);
quit;

/*Ex3 on CALCULATED columns*/

proc sql number;
select CustomerName label="Customer Name", MerchantName label="Merchant Name", amount label="Transaction Amount" format=dollar10.2,
	   datepart(Datetime) as TransactionDate label="Transaction Date" format=date9.
from sq.transactionfull
where month(calculated TransactionDate)=11 or month(calculated TransactionDate) = 12 and Service ne "University";
quit;

/*Exercise s102a06 on DISTINCT clause*/

proc sql number;
select distinct Employed, married  /*obtain all combination of these two columns*/
from sq.customer;
quit;

/*Exercise s102a07 on SUMMARY FUNCTIONS*/

proc sql number;
select count(*) as TotalRows format=comma9., count(Married) as MaritalStatus format=comma9. /*Count--counts ALL row, count()-->counts non missing values*/
from sq.customer;
quit;

/*Exercise s102d04 on HAVING clause*/


proc sql;
select BankID,Employed, count(CustomerID) as TotalCustomers
    from sq.customer
	group by BankID, Employed
	having Totalcustomers >10000
	order by TotalCustomers desc;
quit;

/*Repeat the previous exercise alone!!!*/
title "Bank ordered by TotalCustomers higher then 5000 customers";
proc sql;
select BankID,Employed, count(*) as TotalCustomers
from sq.customer
group by BankID,Employed
having TotalCustomers >5000
order by TotalCustomers desc;
quit;
title;

/*Exercise s102a08 on qtr() and GROUP BY*/

proc sql;
select qtr(datepart(datetime)) as qtr, median(amount) as MedianSpent format=dollar9.2
from sq.transaction
group by qtr;
quit;

/*Exercise s102d05 with boolean columns*/

proc sql ;
create table CustomerCount as
select State, 
       sum(yrdif(DOB,"&SYSDATE9"d) < 25) as Under25 ,sum(yrdif(DOB,"&SYSDATE9"d)>64)as Over64 
    from sq.customer
group by state;
quit;

%put &SYSDATE9;  /*To call the macro variable in the LOG*/

/*Exercise ex4 on count(distinct column)*/

proc sql;
select distinct( CountryCode)
from sq.globalfull
order by CountryCode;
quit;

proc sql;
select count(distinct CountryCode)
from sq.globalfull;
quit;


/*Exercise 5 (with macro program modification)*/
options mcompilenote=all;
options mprint mlogic;

%macro QueryVal;
	proc sql noprint;
	select sum(Amount) as SumAmount, avg(Amount) as AvgAmount format=comma5.2 
	into :SumAmount, :AvgAmount  /*These macro vars are local*/
	from sq.Transactionfull;
	quit;
	%if &SumAmount > 1 %then %do;
	%put NOTE:Amount is high enough!! Sum : &SumAmount Avg: &AvgAmount;
	%end;
	%else %do;
	%put ERROR: Amount is too low!!! Sum: &SumAmount Avg: &AvgAmount;
	%end;
	%put _User_;
	proc sql;
	select customerID, count(*) as NumberTransaction,sum(Amount) as SuspiciousAmount format=comma11.2,(calculated SuspiciousAmount/&SumAmount) as PCTSuspicious 
	from sq.Transactionfull
	where service ne "University" and amount >500
	group by customerID
	order by PCTSuspicious desc;
	quit;
%mend QueryVal;

%QueryVal

/*Exercise s102a10 on creating a table, insert rows,delete a table*/

  /*We create an empty table(with specific columns)*/
proc sql;
create table work.HighCredit
	like sq.customer(keep= FirstName LastName UserID CreditScore);  /*like clause create an empy table with specific columns*/
quit;


  /*We insert rows from another table in work.HighCredit empty table*/

proc sql;
insert into work.highcredit
select FirstName,LastName, UserID, CreditScore
from sq.customer
where creditscore>700;
quit;

  /*Insert in work.HighCredit one row*/


proc sql;
insert into work.highcredit
set FirstName="Hani",
	LastName="Nabulsi",
	UserID="Hani.Nabulsi@gmail.com",
	CreditScore=500;
quit;


proc sql;
select *
from work.highcredit
order by CreditScore;
quit;

   /*Delete the work.HighCredit table*/

proc sql;
drop table work.highcredit;
quit;


/*Exercise s102d06 on managing DICTIONARY tables*/

/*Explore dictionary tables*/

		/*It's a table with all metadata on all tables*/

proc sql number;
describe table dictionary.tables;
select *
from  dictionary.tables
where libname = "SQ";
quit;

		/*SAS equivalent of dictionary.tables*/

proc print data=sashelp.vtable;
where libname= "SQ";
run;

/*Explore dictionary.columns*/			

proc sql number outobs=30;
describe table dictionary.columns;
select *
from dictionary.columns;
quit;

       /*SAS equivalent of dictionary.columns*/

proc print data=sashelp.vcolumn(obs=30);
where libname= "SQ";
run;

/*Explore dictionary.libnames*/

proc sql inobs=30;
describe table dictionary.libnames;
select *
from dictionary.libnames;
quit;

	/*SAS equivalent of dictionary.libnames*/

proc print data=sashelp.vlibnam(obs=30);
run;

/*Ex6 on dictionary.tables*/

proc sql;
title "Number of tables in SQ libname";
select count(*) 
from dictionary.tables
where libname="SQ";
quit;
title;

/*equivalent to...*/


proc sql;
title "Number of tables in SQ libname";
select count(*)
from sashelp.vtable
where libname="SQ";
quit;
title;

/*Ex7 on dictionary.tables*/

proc sql;
title "Number of table for each library";
select libname, count(memname) as TableNumber
from sashelp.vtable
group by libname
order by TableNumber desc;
quit;
title;

/*Exercise s103 on cartesian product*/

/*A  query about ten lines of each table*/

proc sql inobs=10;
title "Table: smallcustomer";
select *
from sq.smallcustomer;
title "Table: smalltransaction";
select *
from sq.smalltransaction;
quit;
title;

/*Cartesian product*/

proc sql;
title "Cartesian product";
select *
from sq.smallcustomer, sq.smalltransaction;
quit;
title;

/*Exercise s103d01 on INNER JOIN*/

/*First 5 rows of SMALLCUSTOMER and SMALLTRANSACTION*/

proc sql inobs=5;
title "Table: SMALLCUSTOMER";
select *
from sq.smallcustomer;
title "Table: SMALLTRANSACTION";
select*
from sq.smalltransaction;
quit;
title;

/*Inner join on ACCOUNTID*/

proc sql number;
select FirstName, LastName, State,Income,Datetime,MerchantID,Amount
from sq.smallcustomer as s inner join sq.smalltransaction as v on s.AccountID = v.AccountID
where state = "NY"
order by amount desc; 
quit;



/*Exercise s103a02 on INNER JOIN*/

proc sql inobs=5;
title "Table: Statepopulation";
select *
from sq.Statepopulation;
title "Table: Statecode";
select *
from sq.Statecode;
quit;

/*Inner join*/
title;
proc sql number;
select name,statename,popEstimate1,popestimate2,popestimate3
from sq.statepopulation as s inner join sq.statecode as v on s.name = v.statecode
order by statename;
quit;

/*Exercise s103a03 on dictionary.columns*/

proc sql number;
select memname as table, name as column
from sashelp.vcolumn
where libname ="SQ" and upcase(name) = "MERCHANTID";
quit;

/*Exercise s103a04 on NON-EQUI JOIN*/

/*First 5 rows*/

proc sql inobs=5;
title "Table: smallcostumer";
select *
from sq.smallcustomer;
title "Table: taxebracket";
select *
from sq.taxbracket;
quit;

/*Inner join non on equal value on a specific variables but if a value is in a range on a specific variable (non-equi join)*/

proc sql number;
select FirstName, LastName, Income format=dollar9.2,TaxBracket
from sq.smallcustomer inner join sq.taxbracket on Income between LowIncome and HighIncome
order by TaxBracket desc, Income desc;  
quit;

/*Another exercise on no-equi join*/

/*First 5 rows*/

proc sql inobs=5;
title "Table: Customer";
select *
from sq.customer;
title"Table: agegroup";
select *
from sq.agegroup;
quit;

title;

/*NO-Equi join*/

proc sql number;
create table work.Person as
select FirstName, LastName, year(dob) as year ,name
from sq.customer inner join sq.agegroup on year(dob) between startyear and endyear;
quit;


/*Exercise s103a05 on LEFT JOIN*/


/*Check first five rows of sq.smallcustomer and sq.smalltransaction*/

proc sql inobs=5;
title "sq.smallcustomer";
select *
from sq.smallcustomer;
title "sq.smalltransaction";
select *
from sq.smalltransaction;
quit;

title;

proc sql number;
select FirstName, LastName, Income , v.AccountID label="Righ AccountID"
from sq.smallcustomer as s left join sq.smalltransaction as v on s.AccountID = v.AccountID; 
quit;

/*Exercise s103d03 on FULL JOIN and COALESCE*/

/*First 5 rows*/

proc sql inobs=5;
title "sq.smallcustomer";
select *
from sq.smallcustomer;
title "sq.smalltransaction";
select *
from sq.smalltransaction;
quit;

title;

/*full join and coalesce()*/

proc sql number;
select firstname, lastname , coalesce(s.accountID, v.accountID) as accountID format=11.,amount
from sq.smallcustomer as s full join sq.smalltransaction as v on s.accountID = v.accountID ;
quit;

/*Exercise s103a06 on managing null value on variable to join*/

/*First 5 rows*/

proc sql inobs=5;
title "sq.smallcustomer";
select *
from sq.smallcustomer2;
title "sq.smalltransaction";
select *
from sq.smalltransaction2;
quit;

title;


/*left join*/

proc sql number;
select merchantID,amount,s.accountid label="left accountID",v.accountid label="right accountID",firstname, lastname
from sq.smalltransaction as s left join sq.smallcustomer as v on s.accountid = v.accountid and s.accountid is not null 
where s.accountID is null;
quit;

/*Exercise s103d04 on REFLEXIVE JOIN*/

/*First 5 rows*/

proc sql inobs=5;
title "Table: sq.employee";
select *
from sq.employee;
quit;

/*REFLEXIVE JOIN*/

proc sql;
select e.employeeID label="Employee ID", e.EmployeeName label="Employee Name", e.JobTitle, e.ManagerID ,m.EmployeeID label="Manager ID", m.EmployeeName label="Manager Name"
from sq.employee as e inner join sq.employee as m on e.managerid = m.EmployeeID
order by m.EmployeeName;
quit;

/*Exercise s103a07 on using substr() on join clause*/

/*Frist 5 rows of sq.transactionfull and sq.statecode*/
proc sql inobs = 5;
title "sq.transactionfull";
select *
from sq.transactionfull;
title "sq.statecode";
select *
from sq.statecode;
quit;


/*Join with substr()*/

proc sql;
select CustomerName, Amount,CustomerID, Income, StateName
from sq.transactionfull as t inner join sq.statecode as s on substr(t.StateID,1,2) = s.statecode;
quit;

/*Exercise s103a08 on join table with put function*/


/*Creat table*/
proc sql;
create table customerzip
	(CustomerID num,
     ZipCode char(5),
     Gender char(1),
     Employed char(1));
insert into customerzip
    values(1,"14580","M","Y")
	values(2,"04429","M","Y")
	values(3,"50101","M","Y")
	values(4,"27519","M","Y")
	values(5,"14216","M","Y")
;
quit;

/*First 5 rows of customerzip and sashelp.zipcode*/

proc sql inobs=5;
title "Table: Customerzip";
select *
from customerzip;
title "Table: sashelp.zipcode";
select *
from sashelp.zipcode;
quit;
title;

/*Inner join with input function*/

proc sql;
select c.customerID, c.zipcode, s.city,s.statename
from customerzip as c inner join sashelp.zipcode as s on input(c.zipcode,z5.) = s.zip;
quit;


/*Exercise s104d01 on subquery*/

/*First 5 rows*/
proc sql inobs=5;
title "sq.statepopulation (outer table)";
select *
from sq.statepopulation;
title "sashelp.us_data (inner table)";
select*
from sashelp.us_data;
quit;

title;


/*Average from 2010 population (inner join)*/
proc sql;
select avg(population_2010) format=comma11.  /*avg: 6,009,064*/
into :avg2010
from sashelp.us_data;
quit;


/*Using subquery*/

proc sql;
title "Region with population higher than 2010 average: &avg2010";
select Name, PopEstimate1 format=comma11., Births1 format= comma10.
from sq.statepopulation
where Popestimate1 > (select avg(population_2010)
					  from sashelp.us_data)
order by PopEstimate1 desc;
quit;

title;

/*Exercise s104a02 subquery with HAVING clause*/

/*Average population for all states*/
proc sql;
select avg(popestimate1) format=comma11.
into :avgPop1
from sq.statepopulation;
quit;

/*Subquery in HAVING clause*/

proc sql;
title "Division with average population higher then average on all States: &avgPop1";
select division , avg(popestimate1) as AvgDiv format=comma11.
from sq.statepopulation
group by division
having AvgDiv > ( select avg(popestimate1) from sq.statepopulation)
order by AvgDiv desc;
quit;



/*Exercise s104d02 on subquery with more values (a whole column!!!)*/


/*inner join(a column as result)*/

proc sql;
select name
into :StateDiv3 separated by ","
from sq.statepopulation
where division = "3";
quit;

proc sql;
title "Customers (&StateDiv3) located in division 3";
create table division3 as
select *
from sq.customer
where State in (select name 
				from sq.statepopulation
				where division = "3");
quit;

/*Exercise s104d03 on IN-LINE VIEWS*/

/*In-line view--> a subquery in sql FROM clause*/

/*First 5 rows*/

proc sql inobs=5;
title "Table: sq.customer";
select FirstName, MiddleName, LastName, State
    from sq.customer;
title "Table: sq.statepopulation";
select Name, EstimateBase
    from sq.statepopulation;
quit;

title;

/*Query with in-line view*/

proc sql number;
select c.state,CustomerNumber,s.EstimateBase, CustomerNumber/s.EstimateBase as PctCustomer format=percent7.3
from (select state, count(*) as CustomerNumber from sq.customer group by state) as c
	  inner join sq.statepopulation as s on c.state = s.name 
order by PctCustomer;	
quit;


/*Exercise s104a04 on VIEW*/

/*Create a simple view*/

proc sql;
create view CustomerNumber as 
select state, count(*) as CustNumber
from sq.customer
group by state;
quit;


/*Call the view*/

proc sql number;
select *
from CustomerNumber  /*Here we call!!*/
order by CustNumber desc;
quit;

/*Plotting*/

proc sgplot data=CustomerNumber;
	hbar State / response=custnumber
                 dataskin=crisp
                 categoryorder=respdesc;
    xaxis label="Total Customer Count";
quit;
title;

/*Exercise s104p06 on a complex view*/

/*Creating the view*/

proc sql;
create view HighRiskCredit as
select zip, sum(avg(creditscore),2*std(creditscore)) as HighRisk
from sq.customer 
where creditscore is not null
group by  zip;
quit;



/*Call the view (in FROM clause)*/

proc sql number ;
title "CUstomer with high credit risk";
select c.FirstName, c.LastName, c.zip format=z5.,c.creditscore,h.highrisk
from sq.customer as c inner join HighRiskCredit as h on c.zip =h.zip
where c.creditscore > h.highrisk
order by c.zip,c.creditscore desc;
quit;

/*Exercise s105a01 on INTERSECT clause (and CORR option)*/

proc sql inobs=5;
title "SQ.SALESEMAIL";
select *
    from sq.salesemail;
title "SQ.SALESPHONE";
select *
    from sq.salesphone;
quit;

proc sql;
title "Customer that respond to email and phone call";
select CustomerID
from sq.salesemail
intersect corr  /*with CORR same result without it*/
select CustomerID
from sq.salesphone;
quit;


/*Exercise s105a02 on EXCEPT*/

/*First 5 rows*/

proc sql inobs=5;
title "SQ.SALESLIST";
select *
    from sq.saleslist;
title "SQ:SALESPHONE";
select *
    from sq.salesphone;
quit;

/*Using EXCEPT*/

proc sql;
title "All customer from Salelist that haven't receved any call from a sale repres.";
select *
from sq.saleslist
except corr /*with corr we can select each column in select statement*/
select *
from sq.salesphone;
quit;

/*Exercise s105d01 on UNION*/

/*first 5 rows*/

proc sql;
title "SQ.SALESMAIL";
select *
    from sq.salesemail;
title "sq.salesphone" ;
select *
    from sq.salesphone;
quit;

proc sql;
title "Customer that have received an email or a representive call";
select count(*)  /*counts the rows from the in-line view (subquery in from clause)*/
from (select *
from sq.salesemail
union corr
select *
from sq.salesphone);
quit;

/*Exercise 15 on EXCEPT clause*/

/*First 5 rows*/

proc sql inobs = 5;
title "sq.merchant";
select *
from sq.merchant;
title "sq.transaction";
select *
from sq.transaction;
quit;

/*Using Except*/

proc sql;
title "Merchant (Shop) without transaction";
select *
from sq.merchant
except corr
select *
from sq.transaction;
quit;

title;

/*Exercise s105d02 on OUTER UNION*/

/*first 5 rows*/

proc sql;
title "sq.salesemail";
select *
from sq.salesemail;
title "sq.salesphone";
select *
from sq.salesphone;
quit;

/*Outer union*/

proc sql;
select *
    from sq.salesemail
      outer union corr
select *
    from sq.salesphone;
quit;

/*equivalent to..*/

data response2;
    /*length Resp $12;*/
    set sq.salesemail/*(rename=(EmailResp=Resp)) */
        sq.salesphone/*(rename=(PhoneResp=Resp))*/;
run;

/*Exercise s106d01 on macro variable in a sql query*/

/*first 5 rows*/

proc sql inobs=5;
title"Table: sq.statepopulation";
select *
from sq.statepopulation;
quit;

/*Put the population avg in a macro var from a sql query*/

proc sql;
select avg(popestimate1) /*format=comma11.*/
into :Avgpop
from sq.statepopulation;
quit;

%put &=Avgpop;

/*Using the macro var*/

proc sql;
title "Nation with population higher than avg: %sysfunc(putn(&Avgpop,comma12.))";
select Name, popEstimate1 format=comma12.
from sq.statepopulation
where popestimate1 > &Avgpop;
quit;

/*Exercise s106a02 on macro var from a sql query*/

/*furst five rows*/

proc sql inobs=5;
title "sq.globalfull";
select *
from sq.globalfull;
quit;

/*macro var from sql query*/

proc sql noprint;
select max(EstYear3pop) format=comma10., count(distinct(countrycode))
into :maxPop trimmed, :CountryNum trimmed
from sq.globalfull;
quit;

%put &maxpop , &CountryNum;

/*Using macro var*/

proc sql;
title "Country with highest 3 year growing population";
title "On a total population of &maxPop";
select CountryCode, shortname, Estyear3pop format=comma13.
from sq.globalfull
where estyear3pop = &maxpop;
quit;

/*Exercise s106d02 on macro var from sql query*/

/*first 5 rows*/

proc sql inobs=5;
title "sq.statepopulation";
select *
from sq.statepopulation;
quit;

/*create macro var from sql query*/

%let division = 9;

proc sql noprint;
select quote(trim(name))
into :CountryDiv9 separated by ","
from sq.statepopulation
where division = "&division";
quit;

%put &=CountryDiv9;

/*Using the macr var*/

options symbolgen;  /*Each time amacro var is called is told into the log with relative value*/

proc sql;
create table Division&division as
select *
from sq.customer
where state in (&CountryDiv9);
quit;

options nosymbolgen;
